In [1]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.metrics import normalized_mutual_info_score
import torch

from utils import *
from autoencoders.basic_autoencoder import BasicAutoencoder
from autoencoders.all_to_all_autoencoder import AllToAllAutoencoder
from autoencoders.transformer_autoencoder import TransformerAutoencoder
from autoencoders.attention_autoencoder import AttentionAutoencoder
from duped_modules.dcn_duped import DCNDuped
from duped_modules.dec_duped import DECDuped
from duped_modules.dec_duped import IDECDuped
from load_datasets import load_all_datasets

In [2]:
device = "cuda"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
np.random.seed(0)

datasets = load_all_datasets(max_rows=5000)
accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}

In [3]:
# Deep Clustering with Basic Autoencoder (no Column Embeddings)
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = len(d.cat_cols) + len(d.cont_cols)
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)
    ae = BasicAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    dcn = DCNDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dcn.fit(d.dataloader)
    nmis[d.name]["No Col Emb DCN"] = normalized_mutual_info_score(d.y, dcn.labels_)
    accuracies[d.name]["No Col Emb DCN"] = cluster_accuracy(d.y, dcn.labels_)

    dec = DECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dec.fit(d.dataloader)
    nmis[d.name]["No Col Emb DEC"] = normalized_mutual_info_score(d.y, dec.labels_)
    accuracies[d.name]["No Col Emb DEC"] = cluster_accuracy(d.y, dec.labels_)

    idec = IDECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    idec.fit(d.dataloader)
    nmis[d.name]["No Col Emb IDEC"] = normalized_mutual_info_score(d.y, idec.labels_)
    accuracies[d.name]["No Col Emb IDEC"] = cluster_accuracy(d.y, idec.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=8, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6, out_features=4, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=4, out_features=2, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=4, out_features=6, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=6, out_features=8, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.157973
Epoch 50/100 - Batch Reconstruction loss: 0.157587
Epoch 75/100 - Batch Reconstruction loss: 0.157757
Epoch 100/100 - Batch Reconstruction loss: 0.156816


/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=7, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=2, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.502609
Epoch 50/100 - Batch Reconstruction loss: 0.440010
Epoch 75/100 - Batch Reconstruction loss: 0.400101
Epoch 100/100 - Batch Reconstruction loss: 0.389916


/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Bank Marketing...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11, out_features=9, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=9, out_features=7, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=7, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=7, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=9, out_features=11, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=11, out_features=13, bias=True)
  (9): LeakyReLU(negative_slope=

...

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Heart Disease...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11, out_features=9, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=9, out_features=7, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=7, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=7, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=9, out_features=11, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=11, out_features=13, bias=True)
  (9): LeakyReLU(negative_slope=0

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Soybean Disease...
Sequential(
  (0): Linear(in_features=35, out_features=30, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=30, out_features=25, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=25, out_features=20, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=20, out_features=15, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=15, out_features=9, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=9, out_features=15, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=15, out_features=20, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=20, out_features=25, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=25, out_features=30, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=30, out_features=35, bias=True)
  (9): LeakyReLU(ne

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

In [4]:
# Deep Clustering with Column Embeddings
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = sum(d for _, d in d.embedding_sizes) + len(d.cont_cols)
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)
    ae = AllToAllAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    dcn = DCNDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dcn.fit(d.dataloader)
    nmis[d.name]["DCN"] = normalized_mutual_info_score(d.y, dcn.labels_)
    accuracies[d.name]["DCN"] = cluster_accuracy(d.y, dcn.labels_)

    dec = DECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dec.fit(d.dataloader)
    nmis[d.name]["DEC"] = normalized_mutual_info_score(d.y, dec.labels_)
    accuracies[d.name]["DEC"] = cluster_accuracy(d.y, dec.labels_)

    idec = IDECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    idec.fit(d.dataloader)
    nmis[d.name]["IDEC"] = normalized_mutual_info_score(d.y, idec.labels_)
    accuracies[d.name]["IDEC"] = cluster_accuracy(d.y, idec.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=9, out_features=7, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=7, out_features=5, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=5, out_features=2, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=7, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.181858
Epoch 50/100 - Batch Reconstruction loss: 0.111451
Epoch 75/100 - Batch Reconstruction loss: 0.097653
Epoch 100/100 - Batch Reconstruction loss: 0.087324


/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11, out_features=9, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=9, out_features=7, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=7, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=7, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=9, out_features=11, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=11, out_features=13, bias=True)
  (9): LeakyReLU(negative_

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Bank Marketing...
Sequential(
  (0): Linear(in_features=22, out_features=18, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=18, out_features=14, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=14, out_features=10, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=10, out_features=6, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=6, out_features=10, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=10, out_features=14, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=14, out_features=18, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=18, out_features=22, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.476802
Epoch 50/100 - Batch Reconstruction loss: 0.516827
Epoch 75/100 - Batch Reconstruction loss: 0.490705
Epoch 100/100 - Bat

...

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Heart Disease...
Sequential(
  (0): Linear(in_features=17, out_features=14, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=14, out_features=11, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=11, out_features=8, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=8, out_features=4, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=8, out_features=11, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=11, out_features=14, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=14, out_features=17, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.596755
Epoch 50/100 - Batch Reconstruction loss: 0.564650
Epoch 75/100 - Batch Reconstruction loss: 0.473070
Epoch 100/100 - Batch Re

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Soybean Disease...
Sequential(
  (0): Linear(in_features=55, out_features=48, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=48, out_features=41, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=41, out_features=34, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=34, out_features=27, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=27, out_features=14, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=14, out_features=27, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=27, out_features=34, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=34, out_features=41, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=41, out_features=48, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=48, out_features=55, bias=True)
  (9): LeakyReLU(

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

In [5]:
# Deep Clustering with Attention
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = len(d.cont_cols) + len(d.cat_cols) * 32
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)
    ae = AttentionAutoencoder(encoder, decoder, d.embedding_sizes, emb_dim=32, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    dcn = DCNDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dcn.fit(d.dataloader)
    nmis[d.name]["Attention DCN"] = normalized_mutual_info_score(d.y, dcn.labels_)
    accuracies[d.name]["Attention DCN"] = cluster_accuracy(d.y, dcn.labels_)

    dec = DECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dec.fit(d.dataloader)
    nmis[d.name]["Attention DEC"] = normalized_mutual_info_score(d.y, dec.labels_)
    accuracies[d.name]["Attention DEC"] = cluster_accuracy(d.y, dec.labels_)

    idec = IDECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    idec.fit(d.dataloader)
    nmis[d.name]["Attention IDEC"] = normalized_mutual_info_score(d.y, idec.labels_)
    accuracies[d.name]["Attention IDEC"] = cluster_accuracy(d.y, idec.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=39, out_features=33, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=33, out_features=27, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=27, out_features=21, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=21, out_features=10, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=10, out_features=21, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=21, out_features=27, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=27, out_features=33, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=33, out_features=39, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.078701
Epoch 50/100 - Batch Reconstruction loss: 0.061157
Epoch 75/100 - Batch Reconstruction loss: 0.058931
Epoch 100/100 - Batch Re

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=193, out_features=175, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=175, out_features=157, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=157, out_features=139, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=139, out_features=121, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=121, out_features=103, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=103, out_features=85, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=85, out_features=67, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=67, out_features=48, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=48, out_features=67, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=67, out_features=85, bias=T

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Bank Marketing...
Sequential(
  (0): Linear(in_features=261, out_features=237, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=237, out_features=213, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=213, out_features=189, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=189, out_features=165, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=165, out_features=141, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=141, out_features=117, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=117, out_features=93, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=93, out_features=65, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=65, out_features=93, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=93, out_features=117, bias=True

...

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Heart Disease...
Sequential(
  (0): Linear(in_features=230, out_features=208, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=208, out_features=186, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=186, out_features=164, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=164, out_features=142, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=142, out_features=120, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=120, out_features=98, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=98, out_features=58, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=58, out_features=98, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=98, out_features=120, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=120, out_features=142, bias=True)
 

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Soybean Disease...
Sequential(
  (0): Linear(in_features=1120, out_features=1036, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1036, out_features=952, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=952, out_features=868, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=868, out_features=784, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=784, out_features=700, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=700, out_features=616, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=616, out_features=532, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=532, out_features=448, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
  (16): Linear(in_features=448, out_features=364, bias=True)
  (17): LeakyReLU(negative_slope=0.01)
  (18): Linear(in_features=364, out_features=280, bias=True)

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

In [6]:
# Deep Clustering with Transformer
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = len(d.cont_cols) + len(d.cat_cols) * 32
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)
    ae = TransformerAutoencoder(encoder, decoder, d.embedding_sizes, emb_dim=32, depth=6, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    dcn = DCNDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dcn.fit(d.dataloader)
    nmis[d.name]["Transformer DCN"] = normalized_mutual_info_score(d.y, dcn.labels_)
    accuracies[d.name]["Transformer DCN"] = cluster_accuracy(d.y, dcn.labels_)

    dec = DECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    dec.fit(d.dataloader)
    nmis[d.name]["Transformer DEC"] = normalized_mutual_info_score(d.y, dec.labels_)
    accuracies[d.name]["Transformer DEC"] = cluster_accuracy(d.y, dec.labels_)

    idec = IDECDuped(n_clusters=d.n_targets, autoencoder=ae, random_state=np.random.RandomState(0))
    idec.fit(d.dataloader)
    nmis[d.name]["Transformer IDEC"] = normalized_mutual_info_score(d.y, idec.labels_)
    accuracies[d.name]["Transformer IDEC"] = cluster_accuracy(d.y, idec.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=39, out_features=33, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=33, out_features=27, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=27, out_features=21, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=21, out_features=10, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=10, out_features=21, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=21, out_features=27, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=27, out_features=33, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=33, out_features=39, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.086698
Epoch 50/100 - Batch Reconstruction loss: 0.059694
Epoch 75/100 - Batch Reconstruction loss: 0.058912
Epoch 100/100 - Batch Re

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=193, out_features=175, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=175, out_features=157, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=157, out_features=139, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=139, out_features=121, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=121, out_features=103, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=103, out_features=85, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=85, out_features=67, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=67, out_features=48, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=48, out_features=67, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=67, out_features=85, bias=T

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Bank Marketing...
Sequential(
  (0): Linear(in_features=261, out_features=237, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=237, out_features=213, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=213, out_features=189, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=189, out_features=165, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=165, out_features=141, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=141, out_features=117, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=117, out_features=93, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=93, out_features=65, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=65, out_features=93, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=93, out_features=117, bias=True

...

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Heart Disease...
Sequential(
  (0): Linear(in_features=230, out_features=208, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=208, out_features=186, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=186, out_features=164, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=164, out_features=142, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=142, out_features=120, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=120, out_features=98, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=98, out_features=58, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=58, out_features=98, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=98, out_features=120, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=120, out_features=142, bias=True)
 

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

Calculating for Soybean Disease...
Sequential(
  (0): Linear(in_features=1120, out_features=1036, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=1036, out_features=952, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=952, out_features=868, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=868, out_features=784, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=784, out_features=700, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Linear(in_features=700, out_features=616, bias=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=616, out_features=532, bias=True)
  (13): LeakyReLU(negative_slope=0.01)
  (14): Linear(in_features=532, out_features=448, bias=True)
  (15): LeakyReLU(negative_slope=0.01)
  (16): Linear(in_features=448, out_features=364, bias=True)
  (17): LeakyReLU(negative_slope=0.01)
  (18): Linear(in_features=364, out_features=280, bias=True)

/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clustering/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/jaanis/repos/attention-mixed-type-clusterin

In [7]:
pd.DataFrame(nmis.values(), index=nmis.keys()).round(4)

,No Col Emb DCN,No Col Emb DEC,No Col Emb IDEC,DCN,DEC,IDEC,Attention DCN,Attention DEC,Attention IDEC,Transformer DCN,Transformer DEC,Transformer IDEC
Abalone,0.1600,0.1573,0.1577,0.1574,0.1585,0.1501,0.1717,0.1630,0.1623,0.1671,0.1642,0.1653
Auction Verification,0.0119,0.0022,0.0081,0.0179,0.0179,0.0179,0.0203,0.0203,0.0203,0.0484,0.0446,0.0444
Bank Marketing,0.0000,0.0000,0.0000,0.0195,0.0195,0.0172,0.0023,0.0023,0.0023,0.0000,0.0000,0.0000
Breast Cancer,0.7081,0.7326,0.7081,0.7333,0.7284,0.7284,0.2185,0.2185,0.2185,0.2062,0.5991,0.2244
Census Income,0.0011,0.0004,0.0000,0.0915,0.0915,0.0911,0.0663,0.0663,0.0663,0.0689,0.0689,0.0689
Credit Approval,0.0098,0.0085,0.0091,0.0024,0.0027,0.0024,0.0000,0.0000,0.0000,0.0036,0.0036,0.0036
Heart Disease,0.1168,0.1202,0.1148,0.1268,0.1273,0.1273,0.1599,0.1599,0.1599,0.1558,0.1494,0.1503
Soybean Disease,0.4327,0.4379,0.4541,0.3930,0.3878,0.3866,0.6306,0.5168,0.6532,0.2605,0.2605,0.2684


In [8]:
pd.DataFrame(accuracies.values(), index=accuracies.keys()).round(4)

,No Col Emb DCN,No Col Emb DEC,No Col Emb IDEC,DCN,DEC,IDEC,Attention DCN,Attention DEC,Attention IDEC,Transformer DCN,Transformer DEC,Transformer IDEC
Abalone,0.1702,0.1659,0.1633,0.1676,0.1621,0.1472,0.1585,0.1286,0.1384,0.1685,0.1549,0.1559
Auction Verification,0.5977,0.5296,0.5835,0.6676,0.6676,0.6676,0.6760,0.6760,0.6760,0.6569,0.6848,0.6843
Bank Marketing,0.6358,0.6274,0.6312,0.5712,0.5712,0.6162,0.7010,0.7010,0.7010,0.7602,0.7602,0.7602
Breast Cancer,0.9517,0.9575,0.9517,0.9473,0.9458,0.9458,0.7467,0.7467,0.7467,0.7350,0.8990,0.7745
Census Income,0.7484,0.5120,0.5184,0.6488,0.6488,0.6486,0.7414,0.7414,0.7414,0.6284,0.6284,0.6284
Credit Approval,0.5651,0.5605,0.5620,0.5360,0.5375,0.5360,0.5299,0.5299,0.5299,0.5467,0.5467,0.5467
Heart Disease,0.3211,0.3077,0.3144,0.3010,0.2977,0.2977,0.3445,0.3445,0.3445,0.3445,0.3378,0.3344
Soybean Disease,0.3203,0.3167,0.3203,0.2669,0.2544,0.2544,0.5125,0.4431,0.5552,0.2278,0.2278,0.2295
